# Caption Analysis

In [1]:
import tiktoken
import pandas as pd
import openai
from time import sleep
import os
file_path = "../../openai_key.txt"

with open(file_path, "r") as file:
    key = file.read()
openai.api_key = key

In [2]:
encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [3]:
# Word Counting Function
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [4]:
def get_questions_anwers(promt):
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user",
         "content": promt}
    ])
    questions_answers = completion.choices[0].message.content
    return questions_answers

In [5]:
def chatgpt(promt):
    success = False
    while not success:
        try:
            qa_pairs = get_questions_anwers(promt)
            success = True
        except:
            print("retrying")
            sleep(5)
    print("-"*50)
    print(qa_pairs)
    return qa_pairs
    

In [6]:
def combined_promt(caption_1, caption_2, requirement):
    
    return f"{requirement}\ncontext 1: {caption_1} \ncontext 2: {caption_2}\noutput:"

def qa_generator(csv_path, demo = True):
    df = pd.read_csv(csv_path)
    if demo:
        df = df[:10]
    df = df[["id", "img", "caption_1", "caption_2"]]

    requirement = "You are required to ask about 10 questions about 2 given contexts. " + \
    "The question topics are: yes/no, counting, object, activity, " + \
    "color, posistion. Answer within 3 words.\n" + \
    "context 1: a man is jumping to catch a ball.\n" + \
    "context 2: a white ball is being catch by a man.\n" + \
    "output:1. What is he doing?\nJumping.\n2. What sport is he playing?\nFootball.\n3. What color is the ball?\nwhite."

    df['promt_3'] = df[["caption_1", "caption_2"]].apply(lambda row: combined_promt(row["caption_1"], row["caption_2"], requirement), axis=1)
    df['qa pairs'] = df['promt_3'].map(chatgpt)

    return df

result = qa_generator("clean_cartoon.csv")

1. Is the wallpaper animated?
Yes.
2. How many people are holding a cup?
Two.
3. What object is in the background?
Microwave.
1. Is the man holding a stick?
Yes.
2. What is the man wearing?
White shirt, blue pants.
3. What objects are on the wall?
Mirror, clock.
retrying


In [ ]:
result        